# Company Network Connectivity: BoardEx Cleaning & Network Construction

In [1]:
import pandas as pd
import numpy as np
import igraph as ig

In [2]:
boardex = pd.read_csv('boardex.csv')

In [3]:
boardex

,associationtype,companyid,overlapyearstart_int,boardname,companyname,directorname,role,associatedrole,conncompanyorgtype,boardid,directorid,roletitle,roleboardposition,roleedflag,startcompanydatestartrole,overlapyearend_int,startcompanydateendrole,conncompanydatestartrole,conncompanydateendrole,orgtype
0,Listed Org,1094770,2004,1 800 CONTACTS INC (De-listed 09/2007),IFS CONSTRUCTION SERVICES LTD (Advanced Ocular...,Steve Newman,Executive Officer (Non-Brd),Independent NED (Brd) (SD),Quoted,3,341621,Executive Officer,Non-Brd,NaN,2002-01-02,2007.0,2007-09-07,2004-04-05,2009-07-28,Quoted
1,Listed Org,13761,1998,1 800 CONTACTS INC (De-listed 09/2007),GRANDVISION (De-listed 03/2004),Edward Butler,Independent Director (Brd) (SD),Chairman (Brd) (SD),Quoted,3,340871,Independent Director,Brd,SD,1998-01-01,1998.0,2007-09-07,1997-01-02,1998-12-31,Quoted
2,Listed Org,28148,2006,1 800 CONTACTS INC (De-listed 09/2007),SITEL CORP (De-listed 01/2007),Steve Key,Independent Director (Brd) (SD),Independent Director (Brd) (SD),Quoted,3,59796,Independent Director,Brd,SD,2005-07-28,2007.0,2007-09-07,2006-08-04,2007-01-31,Quoted
3,Listed Org,40757,2005,1 800 CONTACTS INC (De-listed 09/2007),GREENHILL & CO INC (De-listed 12/2023),Steve Key,Independent Director (Brd) (SD),Independent Director (Brd) (SD),Quoted,3,59796,Independent Director,Brd,SD,2005-07-28,2007.0,2007-09-07,2004-05-01,2020-04-23,Quoted
4,Listed Org,16506,2023,1-800-FLOWERS.COM INC,INTERNATIONAL BUSINESS MACHINES (IBM) CORP,Christina Shim,Director - SD (Brd) (SD),VP/Head of Division (Non-Brd),Quoted,6,2779779,Director - SD,Brd,SD,2023-12-14,2024.0,9000-01-01,2021-11-01,2024-05-02,Quoted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008301,Listed Org,2006211,2023,City Therapeutics Inc,KARYOPHARM THERAPEUTICS INC,Professor Barry Greene,Director - SD (Brd) (SD),Lead Independent Director (Brd) (SD),Quoted,3907518,345845,Director - SD,Brd,SD,1900-01-01,NaN,9000-01-01,2015-01-01,9000-01-01,Private
1008302,Listed Org,2360912,2024,City Therapeutics Inc,VOYAGER THERAPEUTICS INC,Doctor Phil Zamore,Scientific Advisor (Non-Brd),Scientific Advisor (Non-Brd),Quoted,3907518,1273575,Scientific Advisor,Non-Brd,NaN,1900-01-01,NaN,9000-01-01,2020-05-01,9000-01-01,Private
1008303,Listed Org,2147864,2024,City Therapeutics Inc,PROQR THERAPEUTICS NV,Doctor Phil Zamore,Scientific Advisor (Non-Brd),Scientific Advisor (Non-Brd),Quoted,3907518,1273575,Scientific Advisor,Non-Brd,NaN,1900-01-01,NaN,9000-01-01,2017-08-28,9000-01-01,Private
1008304,Listed Org,1986196,2024,RuggedEdge Inc,MCLOUD TECHNOLOGIES CORP (Universal mCloud Cor...,Vincent Higgins,CEO (Brd) (ED),Division President (Non-Brd),Quoted,3915440,881021,CEO,Brd,ED,2024-04-01,2024.0,9000-01-01,2022-01-26,2024-05-22,Private


In [4]:
# Rename columns
boardex = boardex.rename(columns={
    'overlapyearstart_int': 'start_year',
    'overlapyearend_int': 'end_year',
    'boardname': 'company1',
    'companyname': 'company2',
    'directorname': 'person',
    'associatedrole': 'company2_role',
    'roleboardposition': 'company1_role',
    'conncompanyorgtype': 'company2_orgtype',
    'orgtype': 'company1_orgtype',
    'boardid': 'company1_id',
    'companyid': 'company2_id'
})

In [5]:
# Only keep if Board or Non-Board
boardex['company2_role'] = boardex['company2_role'].astype(str)

boardex['company2_role'] = np.where(
    boardex['company2_role'].str.contains(r'\(Brd\)', case=False, na=False),
    'Brd',
    'Non-Brd'
)
boardex = boardex[boardex['company1_role'].isin(['Brd', 'Non-Brd'])]

In [6]:
# Drop columns
boardex.drop(columns=['associationtype','roletitle', 'roleedflag', 'role', 'startcompanydatestartrole', 'startcompanydateendrole', 'conncompanydatestartrole', 'conncompanydateendrole'], inplace=True)

In [7]:
# Re-order columns
new_order = ['start_year', 'end_year', 'company1', 'company2', 'person',
             'company1_role', 'company2_role', 'company1_orgtype', 'company2_orgtype',
             'company1_id', 'company2_id']
boardex = boardex[new_order]

In [8]:
boardex.head(5)

,start_year,end_year,company1,company2,person,company1_role,company2_role,company1_orgtype,company2_orgtype,company1_id,company2_id
0,2004,2007.0,1 800 CONTACTS INC (De-listed 09/2007),IFS CONSTRUCTION SERVICES LTD (Advanced Ocular...,Steve Newman,Non-Brd,Brd,Quoted,Quoted,3,1094770
1,1998,1998.0,1 800 CONTACTS INC (De-listed 09/2007),GRANDVISION (De-listed 03/2004),Edward Butler,Brd,Brd,Quoted,Quoted,3,13761
2,2006,2007.0,1 800 CONTACTS INC (De-listed 09/2007),SITEL CORP (De-listed 01/2007),Steve Key,Brd,Brd,Quoted,Quoted,3,28148
3,2005,2007.0,1 800 CONTACTS INC (De-listed 09/2007),GREENHILL & CO INC (De-listed 12/2023),Steve Key,Brd,Brd,Quoted,Quoted,3,40757
4,2023,2024.0,1-800-FLOWERS.COM INC,INTERNATIONAL BUSINESS MACHINES (IBM) CORP,Christina Shim,Brd,Non-Brd,Quoted,Quoted,6,16506


In [9]:
boardex['company1_role'].value_counts()
boardex['company2_role'].value_counts()

company2_role
Brd        658309
Non-Brd    335874
Name: count, dtype: int64

In [10]:
# Drop rows where both company1_role and company2_role equal 'Non-brd'
boardex = boardex[~((boardex['company1_role'] == 'Non-brd') & (boardex['company2_role'] == 'Non-brd'))]

In [11]:
# Drop duplicate rows based on company1, company2, and person columns, keeping the first occurrence
boardex = boardex.drop_duplicates(subset=['company1', 'company2', 'person'], keep='first')

In [12]:
boardex['company1_orgtype'].value_counts()

company1_orgtype
Quoted         542692
Private         81594
Partnership     21603
Government       1725
Charities         430
Clubs              16
Name: count, dtype: int64

In [13]:
# Drop rows where orgtype not quoted
boardex = boardex[~boardex['company1_orgtype'].isin(['Charities', 'Clubs', 'Government', 'Partnership', 'Private'])]

In [15]:
boardex['company1_orgtype'].value_counts()

company1_orgtype
Quoted    542692
Name: count, dtype: int64

In [14]:
boardex['company2_orgtype'].value_counts()

company2_orgtype
Quoted    542692
Name: count, dtype: int64

In [ ]:
boardex_df = boardex

def expand_years(row):
    # If start_year is missing, we cannot expand, so assign None to 'year'
    if pd.isna(row['start_year']):
        new_row = row.copy()
        new_row['year'] = None
        return pd.DataFrame([new_row])

    start_year = int(row['start_year'])

    # If end_year is missing, use 2025; otherwise, convert end_year to int.
    if pd.isna(row['end_year']):
        end_year = 2025
    else:
        end_year = int(row['end_year'])

    # Create a row for each year in the range from start_year to end_year (inclusive)
    new_rows = []
    for year in range(start_year, end_year + 1):
        new_row = row.copy()
        new_row['year'] = year
        new_rows.append(new_row)

    return pd.DataFrame(new_rows)

expanded_df = pd.concat([expand_years(row) for _, row in boardex_df.iterrows()], ignore_index=True)
print(expanded_df.head())

In [21]:
# Save to CSV
expanded_df.to_csv("expanded_df.csv", index=False)

In [17]:
# Import CSV
expanded_df = pd.read_csv('expanded_df.csv')

In [19]:
# Filter out non-quoted companies
expanded_df = expanded_df[expanded_df['company1_orgtype'] == "Quoted"]

In [21]:
expanded_df['year'] = pd.to_datetime(expanded_df['year'], format='%Y-%m-%d').dt.year

/var/folders/87/l9v54pbn62z8ml1qb5cc92ch0000gp/T/ipykernel_56227/1554290954.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  expanded_df['year'] = pd.to_datetime(expanded_df['year'], format='%Y-%m-%d').dt.year


In [22]:
expanded_df

,start_year,end_year,company1,company2,person,company1_role,company2_role,company1_orgtype,company2_orgtype,company1_id,company2_id,year
0,2004,2007.0,1 800 CONTACTS INC (De-listed 09/2007),IFS CONSTRUCTION SERVICES LTD (Advanced Ocular...,Steve Newman,Non-Brd,Brd,Quoted,Quoted,3,1094770,2004
1,2004,2007.0,1 800 CONTACTS INC (De-listed 09/2007),IFS CONSTRUCTION SERVICES LTD (Advanced Ocular...,Steve Newman,Non-Brd,Brd,Quoted,Quoted,3,1094770,2005
2,2004,2007.0,1 800 CONTACTS INC (De-listed 09/2007),IFS CONSTRUCTION SERVICES LTD (Advanced Ocular...,Steve Newman,Non-Brd,Brd,Quoted,Quoted,3,1094770,2006
3,2004,2007.0,1 800 CONTACTS INC (De-listed 09/2007),IFS CONSTRUCTION SERVICES LTD (Advanced Ocular...,Steve Newman,Non-Brd,Brd,Quoted,Quoted,3,1094770,2007
4,1998,1998.0,1 800 CONTACTS INC (De-listed 09/2007),GRANDVISION (De-listed 03/2004),Edward Butler,Brd,Brd,Quoted,Quoted,3,13761,1998
...,...,...,...,...,...,...,...,...,...,...,...,...
2705442,2024,NaN,POWER SOLUTIONS INTERNATIONAL INC,EXRO TECHNOLOGIES INC,Frank Simpkins,Brd,Brd,Quoted,Quoted,3906438,2696370,2025
2705443,2024,NaN,POWER SOLUTIONS INTERNATIONAL INC,FERRETTI SPA,Kevin Jiang,Brd,Brd,Quoted,Quoted,3906438,3522838,2024
2705444,2024,NaN,POWER SOLUTIONS INTERNATIONAL INC,FERRETTI SPA,Kevin Jiang,Brd,Brd,Quoted,Quoted,3906438,3522838,2025
2705445,2024,NaN,POWER SOLUTIONS INTERNATIONAL INC,KION GROUP AG,Kevin Jiang,Brd,Brd,Quoted,Quoted,3906438,1967313,2024


### Calculate Network Centrality Metrics

In [23]:
results_list = []

# Group the DataFrame by 'year'
for yr, df_year in expanded_df.groupby('year'):
    # Get the unique company IDs using pd.concat and .unique()
    companies = sorted(pd.concat([df_year['company1_id'], df_year['company2_id']]).unique())
    
    # Create a mapping from company id to index (for igraph)
    mapping = {company: idx for idx, company in enumerate(companies)}
    
    # Create sorted pairs using vectorized numpy operations
    comp1 = df_year['company1_id'].values
    comp2 = df_year['company2_id'].values
    # For numeric or comparable types, np.minimum and np.maximum work well:
    u = np.minimum(comp1, comp2)
    v = np.maximum(comp1, comp2)
    pairs = list(zip(u, v))
    
    # Use pandas value_counts to compute edge weights
    edge_weights = pd.Series(pairs).value_counts().to_dict()
    
    # Create an igraph Graph and add vertices with the company IDs
    g = ig.Graph()
    g.add_vertices(companies)
    
    # Prepare edges (as indices) and weights for the graph
    edges = [(mapping[u], mapping[v]) for (u, v), w in edge_weights.items()]
    weights = [w for (u, v), w in edge_weights.items()]
    
    # Add edges and set their 'weight' attribute
    g.add_edges(edges)
    g.es['weight'] = weights
    
    # Calculate network centrality metrics
    inv_weights = [1.0 / w for w in weights]
    betweenness = g.betweenness(weights=inv_weights)
    closeness = g.closeness()  # unweighted closeness
    n = g.vcount()
    degree_counts = g.degree()
    degree = [d / (n - 1) if n > 1 else 0 for d in degree_counts]
    
    try:
        eigenvector = g.eigenvector_centrality(weights="weight")
    except Exception as e:
        eigenvector = [None] * n
    
    df_metrics = pd.DataFrame({
        'company_linking': g.vs['name'],
        'year': yr,
        'betweenness': betweenness,
        'closeness': closeness,
        'degree': degree,
        'eigenvector': eigenvector
    })
    
    results_list.append(df_metrics)

metrics_df = pd.concat(results_list, ignore_index=True)
print(metrics_df)


        company_linking  year   betweenness  closeness    degree   eigenvector
0                  6930  1960      0.000000   1.000000  1.000000  1.000000e+00
1                 24708  1960      0.000000   1.000000  1.000000  1.000000e+00
2                  6930  1961      0.000000   1.000000  1.000000  1.000000e+00
3                 24708  1961      0.000000   1.000000  1.000000  1.000000e+00
4                  6930  1962      0.000000   1.000000  0.166667  0.000000e+00
...                 ...   ...           ...        ...       ...           ...
312928          3903002  2025   8788.008820   0.182885  0.000313  2.724054e-12
312929          3904801  2025  12003.373911   0.162327  0.000940  1.278264e-17
312930          3905369  2025      0.000000   0.201240  0.000209  1.715523e-09
312931          3906385  2025  27476.946429   0.176810  0.000731  1.728229e-13
312932          3906438  2025   9774.501511   0.170673  0.000313  1.420494e-11

[312933 rows x 6 columns]


In [24]:
metrics_df.to_csv("metrics_df_run2.csv", index=False)